# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева, НИУ ВШЭ*

## Управление браузером с Selenium: ВКонтакте и скроллинг

### Часть 1: залогиниваемся во ВКонтакте

Импортируем необходимые компоненты Selenium:

In [1]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

Открываем браузер – начинаем новую сессию работы с Selenium:

In [2]:
br = wd.Chrome()

Переходим в браузере на главную страницу ВКонтакте:

In [3]:
br.get("https://vk.com/")

Находим поле для логина:

In [4]:
login = br.find_element(By.ID, "index_email")

Отправляем туда свой логин:

In [5]:
login.send_keys("allatambov@mail.ru")

Чтобы не искать на странице кнопку Войти (не очень удобно), просто имитируем нажатие кнопки *Enter*:

In [6]:
login.send_keys(Keys.ENTER)

Тут придётся вручную ввести код из сообщения, всё равно Python его автоматически из смс-ки не считает. Будем считать, что залогинились. Можно искать информацию!

### Часть 2: ищем пользователей

Перейдём на страницу поиска друзей:

In [7]:
br.get("https://vk.com/friends")

Найдём на этой странице ссылку с текстом *Поиск друзей*, чтобы перейти к странице поиска пользователей по заданным критериям, и кликнем на неё:

In [8]:
search = br.find_element(By.LINK_TEXT, "Поиск друзей")
search.click()

Чтобы формировать критерии для поиска, нам нужно открыть меню с фильтрами – развернуть меню *Параметры поиска*. Изучив исходный код страницы, видим, что параметры поиска можно найти по id:

In [9]:
pars = br.find_element(By.ID, "friends_filters_block")
pars.click()

Для начала выберем город. В исходном коде страницы поле для ввода города имеет тэг `<input>` и id, равный `cCity`:

In [10]:
city = br.find_element(By.ID, "cCity")

Объект `city` – это объект типа `webelement.WebElement`, то есть элемент страницы, с которым умеет работать модуль `webdriver` из Selenium. По такому объекту тоже можно выполнять поиск с помощью метода `.find_element()`.

В нашем случае внутри этого элемента нужно найти другой, с тэгом `<input>`, потому что нас интересует поле для ввода значения. Если этим шагом пренебречь и попытаться ввести название страны прямо в `city`, мы получим ошибку вида `element not interactable`, потому что сам по себе раздел со страной никакого взаимодействия с пользователем не предполагает, его нельзя редактировать, на него нельзя кликать и прочее.

Поэтому найдём внутри `city` поле для ввода значения по тэгу:

In [11]:
city_inp = city.find_element(By.TAG_NAME, "input")
city_inp.send_keys("Москва")

Отлично! Значение выбрано. Но есть проблема – оно «повисло» в воздухе, опция с выбором страны отображается как выбранная в выпадающем меню, но в самом поле выбор не зафиксирован. Чтобы подтвердить выбор, нужно нажать на клавишу *Enter*:

In [12]:
br.implicitly_wait(3)  # задержка 3 секунды
city_inp.send_keys(Keys.ENTER)

А вот с полом всё поинтереснее: найти поле для ввода пола просто, а вот значения нужно выбирать, нажимая на радиокнопки (*radiobuttons*). Сначала найдём поле для выбора пола:

In [13]:
sex = br.find_element(By.ID, "cSex")

А теперь – все опции внутри (согласно исходному коду, они имеют тэг `<div>`):

In [14]:
values = sex.find_elements(By.TAG_NAME, "div")
values

[<selenium.webdriver.remote.webelement.WebElement (session="fe4ca10e3d69721a6a92cae732f83f20", element="FD7265A8F9BEA42FBFC55D2D2517CBC5_element_168")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fe4ca10e3d69721a6a92cae732f83f20", element="FD7265A8F9BEA42FBFC55D2D2517CBC5_element_169")>,
 <selenium.webdriver.remote.webelement.WebElement (session="fe4ca10e3d69721a6a92cae732f83f20", element="FD7265A8F9BEA42FBFC55D2D2517CBC5_element_170")>]

Обратите внимание: здесь метод `.find_elements()`, не `.find_element()`, потому что результатов ожидается несколько. Выбираем мужской пол – это второй элемент списка – и кликаем на него:

In [15]:
values[1].click()

На этом закончим работу с фильтрами и перейдём к результатам. На этом этапе возможности Selenium нам пока не понадобятся, нам нужно только запросить исходный код страницы, которая сейчас открыта в окне браузера, управляемом из Python, и продолжить работу с HTML с помощью BeautifulSoup.

In [16]:
html = br.page_source

Импортируем BeautifulSoup:

In [17]:
from bs4 import BeautifulSoup

Преобразуем HTML в `soup`:

In [18]:
soup = BeautifulSoup(html)

Создадим список `divs` с фрагментами html-кода, каждый из которых соответствует одной «карточке» пользователя с именем и ссылкой на профиль.

In [19]:
divs = soup.find_all("div", {"class" : "labeled name"})

Напишем функцию `get_person()` для извлечения имени и ссылки на профиль пользователя:

In [20]:
def get_person(d):
    name = d.text
    href = d.find("a")["href"]
    full_href = "https://vk.com" + href
    return name, full_href

Применим функцию ко всем элементам в `divs`:

In [21]:
people = [get_person(d) for d in divs] 
people

[('Дмитрий Ефимов', 'https://vk.com/dbefimov'),
 ('Михаил Пензин', 'https://vk.com/charodall'),
 ('Madjid Tashpulatov', 'https://vk.com/id573663822'),
 ('Иван Анкудинов', 'https://vk.com/vanyank'),
 ('Эмир Юнусов', 'https://vk.com/id470787305'),
 ('Михаил Журавлев', 'https://vk.com/catt_inside'),
 ('Вадим Анисимов', 'https://vk.com/vadumus'),
 ('Семён Маляров', 'https://vk.com/fishcatfish'),
 ('Андрей Савельев', 'https://vk.com/aasavelev'),
 ('Иоанн Довгополый', 'https://vk.com/dereinzigemitdemeigentum'),
 ('Андрей Шульга', 'https://vk.com/shulga_andrei'),
 ('Михаил Новосёлов', 'https://vk.com/newoselove'),
 ('Герман Быков', 'https://vk.com/phenomen_sage'),
 ('Антон Айсин', 'https://vk.com/anton.aisin'),
 ('Александр Стоев', 'https://vk.com/stoevkaiser'),
 ('Ярослав Кандыба', 'https://vk.com/hangareighteen'),
 ('Максим Мицкевич', 'https://vk.com/mitskm'),
 ('Никита Зарипов', 'https://vk.com/letsgowithyou'),
 ('Иван Куницын', 'https://vk.com/kunitssyn'),
 ('Олег Антопкин', 'https://vk.c

### Часть 3: скроллинг

Осталась одна проблема – мы извлекли далеко не все результаты, а только те результаты, которые «видит» браузер, то есть то, что мы как пользователи видим до скроллинга. Библиотека Selenium умеет скроллить страницы, точнее, активировать запуск кода на JavaScript, который отвечает за скроллинг. В общем виде строка с кодом для скроллинга выглядит так (Y – на сколько пикселей нужно проскроллить):

    br.execute_script("window.scrollTo(0, Y)") 

Если нужно проскроллить до конца страницы, то тогда вместо `Y` нужно вписать значение, которое извлекается из тела документа HTML:

    document.body.scrollHeight

Например, проскроллим текущую страницу до самого низа:

In [22]:
br.execute_script("window.scrollTo(0, document.body.scrollHeight);")

Далее, чтобы открыть новые результаты, нам нужно найти кнопку *Показать ещё*. Если посмотрим внимательно на исходный код, найдём id этой кнопки:

In [23]:
button_more = br.find_element(By.ID, "ui_search_load_more")

Кликаем:

In [24]:
button_more.click()

Теперь нам нужно сделать следующее: выгружать информацию из исходного кода страницы по новым пользователям и скроллить страницу дальше. И повторять эти действия до тех пор, пока результаты не закончатся. Для начала напишем функцию `get_users_info()`, которая принимает на вход объект `br` (окно браузера), считывает исходный код и возвращает список с «чистыми» результатами – список пар (имя пользователя, ссылка на профиль).

### Задача 1

Напишите функцию `get_users_info()` согласно описанию выше.

In [25]:
def get_users_info(br):
    html = br.page_source
    soup = BeautifulSoup(html)
    divs = soup.find_all("div", {"class" : "labeled name"})
    people = [get_person(d) for d in divs] 
    return people

Теперь напишем код (адаптированный ответ с [StackOverflow](https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python)) для цикла. Какой тип цикла нам нужен? Нам нужен цикл, который умеет повторять операции до тех пор, пока мы не дойдём до последнего результата, то есть до того момента, когда скроллить будет некуда. Воспользуемся конструкцией `while True`, бесконечным вариантом цикла `while`, который будет запускаться до тех пор, пока не дойдёт до кода с оператором `break` (выход из цикла) или не столкнётся с ошибкой. 

In [26]:
from time import sleep

In [27]:
# результатов много, фильтры очень общие
# оставновим цикл сами через Стоп (Kernel – Interrupt)

all_results = []
last_height = br.execute_script("return document.body.scrollHeight")

while True:
    res = get_users_info(br)
    all_results.extend(res) 
    
    br.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    new_height = br.execute_script("return document.body.scrollHeight")
    sleep(1)
    
    if new_height == last_height:
        break
        
    last_height = new_height

KeyboardInterrupt: 

In [28]:
all_results[0:15]

[('Дмитрий Ефимов', 'https://vk.com/dbefimov'),
 ('Михаил Пензин', 'https://vk.com/charodall'),
 ('Madjid Tashpulatov', 'https://vk.com/id573663822'),
 ('Иван Анкудинов', 'https://vk.com/vanyank'),
 ('Эмир Юнусов', 'https://vk.com/id470787305'),
 ('Михаил Журавлев', 'https://vk.com/catt_inside'),
 ('Вадим Анисимов', 'https://vk.com/vadumus'),
 ('Семён Маляров', 'https://vk.com/fishcatfish'),
 ('Андрей Савельев', 'https://vk.com/aasavelev'),
 ('Иоанн Довгополый', 'https://vk.com/dereinzigemitdemeigentum'),
 ('Андрей Шульга', 'https://vk.com/shulga_andrei'),
 ('Михаил Новосёлов', 'https://vk.com/newoselove'),
 ('Герман Быков', 'https://vk.com/phenomen_sage'),
 ('Антон Айсин', 'https://vk.com/anton.aisin'),
 ('Александр Стоев', 'https://vk.com/stoevkaiser')]

Пояснения к коду.

1. Сохраняем в переменную `last_height` величину, на которую мы можем проскроллить страницу за один раз в данный момент времени, чтобы дойти до конца страницы. 

2. На каждой итерации цикла `while` мы выгружаем информацию, добавляем её в список `all_results` и скроллим страницу до самого низа. После скроллинга проверяем, на сколько ещё можно проскроллить, сохраняем полученное значение в `new_height`. 

3. Если скроллить уже некуда, если мы находимся в самом низу «бесконечной» страницы с результатами поиска, то `new_height` совпадает с `last_height`. Значит, нам нужно остановить исполнение кода – выходим из цикла с помощью `break`. 

4. Если мы ещё не закончили скроллить, обновляем значение `last_height`, заменяя его на `new_height` (теперь уже в нём хранится величина, на которую мы можем проскроллить страницу за один раз в данный момент времени). Продолжаем выполнять выгрузку информации и скроллинг.

### Задача 2

Создайте новый список `results_unique`, который не содержит повторений (набор уникальных пар значений).

In [29]:
results_unique = list(set(all_results)) 

In [30]:
print(len(results_unique))

620


### Задача 3

Создайте датафрейм на основе списка `results_unique`, присвойте столбцам подходящие названия и выгрузите таблицу в файл Excel.

In [31]:
import pandas as pd

In [32]:
df = pd.DataFrame(results_unique)
df.columns = ["name", "link"]

# 10 строк для примера
df.head(10)

,name,link
0,Василий Виноградов,https://vk.com/id324192335
1,Рома Частин,https://vk.com/romanchastin
2,Антон Гребенников,https://vk.com/whatmeworry
3,Алексей Нестеренко,https://vk.com/id132319
4,Артур Магамалиев,https://vk.com/mr_incognito777
5,Элвин Грей,https://vk.com/elvingrey
6,Григорий Хвацкий,https://vk.com/ax0lotl
7,Даулет Жанайдаров,https://vk.com/pripev_dva_raza
8,Антон Айсин,https://vk.com/anton.aisin
9,Миша Бутков,https://vk.com/misha_butkov


In [33]:
df.to_excel("users.xlsx")